## Run a job from inside a Domino Workspace

You need a token to invoke a job in Domino. 
- You can use a DOMINO_USER_API_KEY from the workspace environment variable
- You can get a new access token each time you need to make a domino API call. The token is valid for 5 mins

Find the docs for python based client [here](https://github.com/dominodatalab/python-domino) or endpoint based client [here](https://github.com/dominodatalab/python-domino)

In [1]:
import os

In [14]:
import requests
def get_access_token():
    api_proxy=os.environ["DOMINO_API_PROXY"]
    x = requests.get(f"{api_proxy}/access-token")
    if x.status_code==200:
        return x.text
    return ''

In [17]:
##Using API Key
from domino import Domino
# By and large your commands will run against a single project,
# so you must specify the full project name
project_owner=os.environ['DOMINO_PROJECT_OWNER']
project_name=os.environ['DOMINO_PROJECT_NAME']
domino_1 = Domino(f"{project_owner}/{project_name}",api_key=os.environ['DOMINO_USER_API_KEY'])
domino_2 = Domino(f"{project_owner}/{project_name}",auth_token=get_access_token())


In [18]:
##Using API KEY
print("User Id " +  domino_1.get_user_id(project_owner))
##Using Access Token
print("User Id " +  domino_2.get_user_id(project_owner))

User Id 6283a3966d4fd0362f8ba2a8
User Id 6283a3966d4fd0362f8ba2a8


### A General Rule of Thumb
1. Use the API Key when invoking the Domino API from outside Domino
2. Use the Access Token when invoking the Domino API key from inside a Domino Workload

## Running Jobs using Domino API from inside a workspace

1. You have to commit your code first. A job is a separate workload
2. Do not pass sensitive information to Jobs. Instead update a user envirionment variable before starting a job and read it from inside a job